In [1]:
import pandas as pd
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter
import numpy as np
from sklearn.cluster import KMeans

In [2]:
USE_ONLY_OS = False
USE_ONLY_EFS = False
SEED = 42
np.random.seed(SEED)

HEADERS = ["Patient_ID","EPAS1","ERC2","PRC1","CSGALNACT1","CCND1","OS_event","OS_time","EFS_event","EFS_time", "ISS Staging"]
dataset = pd.read_csv("./final_columns.csv")

In [3]:
parameters = [
    "EPAS1",
    "ERC2",
    "PRC1",
    "CSGALNACT1",
    "CCND1"
    ]
OS_TIME_COL = "OS_time"
EFS_TIME_COL = "EFS_time"
OS_EVENT_COL = "OS_event"
EFS_EVENT_COL = "EFS_event"
times = [OS_TIME_COL, EFS_TIME_COL]

best = {}
os_event = dataset[OS_EVENT_COL].to_numpy()
efs_event = dataset[EFS_EVENT_COL].to_numpy()

for param in parameters:
    best[param] = {}
    min_p_os = np.inf
    min_p_pfs = np.inf
    x = dataset[param].to_numpy()
    uniq = np.unique(x)    #sorted list of unique non-missing values
    if uniq.size<2:
        continue
    for u in uniq:
        left = x<=u
        right = x>u
        n_left, n_right = int(left.sum()), int(right.sum())
        if n_left == 0 or n_right == 0:
            continue

        for tm in times:
            if tm == OS_TIME_COL:  e = os_event
            if tm == EFS_TIME_COL:  e = efs_event
            
            t = dataset[tm].to_numpy()
            res = logrank_test(
                t[left], t[right],
                event_observed_A=e[left],
                event_observed_B=e[right]
            )
            stat = float(res.test_statistic)
            pval = float(res.p_value)
            if tm == OS_TIME_COL:
                if pval<=min_p_os:
                    min_p_os = pval
                    best[param]["OS"] = u
                    best[param]["OS_p"] = pval
            elif tm == EFS_TIME_COL:
                if pval<=min_p_pfs:
                    min_p_pfs = pval
                    best[param]["EFS"] = u
                    best[param]["EFS_p"] = pval

In [4]:
THRESHOLD = {}

if USE_ONLY_OS:
    print("USING ONLY OS")
    for param in best.keys():
        THRESHOLD[param] = best[param]["OS"]
elif USE_ONLY_EFS:
    print("USING ONLY EFS")
    for param in best.keys():
        THRESHOLD[param] = best[param]["EFS"]
else:
    print("parameter\tValue\tp-value")
    print(best)
    for param in best.keys():
        EFS_IS_BETTER = best[param]["EFS_p"] < best[param]["OS_p"]
        print()
        print(param)
        print("EFS:\t\t", best[param]["EFS"], "\t", best[param]["EFS_p"], "\t<=====BEST====="*EFS_IS_BETTER)
        print("OS:\t\t", best[param]["OS"], "\t", best[param]["OS_p"], "\t<=====BEST====="*(not EFS_IS_BETTER))
        THRESHOLD[param] = best[param]["EFS"] if best[param]["EFS_p"] < best[param]["OS_p"] else best[param]["OS"]
        THRESHOLD[param] = float(THRESHOLD[param])
print(THRESHOLD)

parameter	Value	p-value
{'EPAS1': {'OS': np.float64(6.9115), 'OS_p': 1.2534870841992595e-07, 'EFS': np.float64(8.6686), 'EFS_p': 6.5949289368758665e-09}, 'ERC2': {'OS': np.float64(6.8081), 'OS_p': 0.00034390488424502374, 'EFS': np.float64(2.113), 'EFS_p': 0.00018642895813774827}, 'PRC1': {'OS': np.float64(10.1829), 'OS_p': 4.6752251831095895e-08, 'EFS': np.float64(4.4355), 'EFS_p': 1.2731087481035051e-14}, 'CSGALNACT1': {'OS': np.float64(10.545), 'OS_p': 4.229187500112521e-05, 'EFS': np.float64(8.9912), 'EFS_p': 0.0020814814931195474}, 'CCND1': {'OS': np.float64(3.2879), 'OS_p': 2.298789099495099e-07, 'EFS': np.float64(3.4128), 'EFS_p': 4.0035292978524564e-05}}

EPAS1
EFS:		 8.6686 	 6.5949289368758665e-09 	<=====BEST=====
OS:		 6.9115 	 1.2534870841992595e-07 

ERC2
EFS:		 2.113 	 0.00018642895813774827 	<=====BEST=====
OS:		 6.8081 	 0.00034390488424502374 

PRC1
EFS:		 4.4355 	 1.2731087481035051e-14 	<=====BEST=====
OS:		 10.1829 	 4.6752251831095895e-08 

CSGALNACT1
EFS:		 8.9912 

In [5]:
cph = CoxPHFitter()
cph.fit(dataset[[OS_TIME_COL, OS_EVENT_COL] + parameters], duration_col=OS_TIME_COL, event_col=OS_EVENT_COL)
hr_os = cph.hazard_ratios_.rename("OS")

cph.fit(dataset[[EFS_TIME_COL, EFS_EVENT_COL] + parameters], duration_col=EFS_TIME_COL, event_col=EFS_EVENT_COL)
hr_efs = cph.hazard_ratios_.rename("EFS")

hr_df = pd.concat([hr_os, hr_efs], axis=1)

if USE_ONLY_OS:
    print("Using only OS")
    hr_df["Higher_HR"] = hr_df["OS"]
elif USE_ONLY_EFS:
    print("Using only PFS")
    hr_df["Higher_HR"] = hr_df["EFS"]
else:
    print("Using both OS & PFS")
    hr_df["Higher_HR"] = hr_df.max(axis=1)

NEW_MIN = 1
NEW_MAX = 4

hr_df["normalized"] = 1 + (hr_df["Higher_HR"] - hr_df["Higher_HR"].min())/(hr_df["Higher_HR"].max() - hr_df["Higher_HR"].min()) * (NEW_MAX - NEW_MIN)

print("=== Hazard Ratios from OS vs EFS ===")
print(hr_df)

weights = hr_df["normalized"].to_dict()
weights

Using both OS & PFS
=== Hazard Ratios from OS vs EFS ===
                  OS       EFS  Higher_HR  normalized
covariate                                            
EPAS1       0.782848  0.754122   0.782848    1.000000
ERC2        0.883005  0.951248   0.951248    1.943442
PRC1        1.318333  1.185434   1.318333    4.000000
CSGALNACT1  0.775049  0.849147   0.849147    1.371433
CCND1       0.934810  0.918160   0.934810    1.851350


{'EPAS1': 1.0,
 'ERC2': 1.9434423366919555,
 'PRC1': 4.0,
 'CSGALNACT1': 1.3714330380236575,
 'CCND1': 1.851349857009883}

In [6]:

"""
    Among them, EPAS1,
    ERC2, CSGALNACT1, and CCND (hazard ratio <1) were
    protective genes, while PRC1 was a harmful gene (hazard ratio
    >1). [Page 4]
"""
high_risk_side = {
    "EPAS1": "<",
    "ERC2": "<",
    "PRC1": ">",
    "CSGALNACT1": "<",
    "CCND1": "<"
}

scores_train = pd.Series(0.0, index=dataset.index, name="risk_score")

for p in parameters:
    thr  = THRESHOLD[p]
    w    = weights[p]
    side = high_risk_side[p]

    col_tr = dataset[p]

    if side == '>':
        mask_tr = col_tr > thr
    else:
        mask_tr = col_tr < thr

    scores_train += w * mask_tr.astype(float)


In [7]:

def map_by_median(labels, scores, prefix="GENEI-"):
    """
    Map arbitrary cluster labels to ordered labels (e.g., MRS-1..MRS-k)
    based on ascending cluster median of `scores`.
    Ties are broken deterministically by the original cluster id.
    """
    
    labels = pd.Series(labels, index=scores.index if isinstance(scores, pd.Series) else None)
    uniq = np.unique(labels)

    # compute medians per cluster
    med = {k: float(scores[labels == k].median()) for k in uniq}

    # sort by (median, cluster_id) to break ties deterministically
    order = sorted(uniq, key=lambda k: (med[k], k))

    # build mapping: lowest median -> prefix-1, highest -> prefix-k
    rank_map = {k: f"{prefix}-{i+1}" for i, k in enumerate(order)}
    return pd.Series([rank_map[l] for l in labels], index=labels.index), med

rep_train = scores_train.values.reshape(-1, 1)

kmeans = KMeans(n_clusters=3, random_state=SEED, n_init=10)
labels_train = kmeans.fit_predict(rep_train)

risk_train, med = map_by_median(labels_train, scores_train, prefix="GENEI-")


train_labeled = dataset.copy()
train_labeled["risk_score"] = scores_train
train_labeled["risk_group"] = risk_train

# Map TEST raw ids using the TRAIN order so names are consistent
order_ids = sorted(np.unique(labels_train), key=lambda k: (med[k], k))
rank_map = {k: f"GENEI-{i+1}" for i, k in enumerate(order_ids)}

print(train_labeled["risk_group"].value_counts().sort_index())

risk_group
GENEI--1    199
GENEI--2    284
GENEI--3     76
Name: count, dtype: int64


In [8]:
dataset["ISS Staging"] = dataset["ISS Staging"].astype(str).replace({"nan": "Unknown"})
if dataset["ISS Staging"].isna().any():
    dataset["ISS Staging"] = dataset["ISS Staging"].fillna("Unknown")

out_cols = ["Patient_ID"] + parameters + ["risk_score", "risk_group", "ISS Staging", "OS_event", "OS_time", "EFS_event", "EFS_time"]
labeled = train_labeled[out_cols].copy()

labeled.to_csv("full_GENEI.csv", index=False)
print("Saved labels to genei_labels_full_cohort.csv")

Saved labels to genei_labels_full_cohort.csv
